Sample package
===============

My own sample R package implementing sumx function with rPython library: https://github.com/pprzetacznik/pythonTestPackage

deepLearnR samples
=============

In [12]:
library(deepLearnR)

{
library(MASS)
data(Boston)
X <- Boston[,2:14]
y <- Boston[,1]
TensorFlowDNNRegressor(modelTag="tfdnnr-01", X=X, y=y, steps=5000)
pred <- TensorFlow.regressorEval(modelTag="tfdnnr-01")
mse <- rPython::python.get("mse")
r2 <- rPython::python.get("r2")
}

In [13]:
r2

[1] -1.077636

In [14]:
{
x.vals <- seq(1:100)
y.vals <- 0.3 + 0.5 * x.vals
lm.tf.fit <- deepLearnR::TensorFlow.SystemLinReg(X = x.vals, Y = y.vals,
epochs = 100000, learning.rate = .00005)
lm.tf.fit
}

$slope
[1] 0.5004605

$intercept
[1] 0.2691519

In [15]:
Y <- deepLearnR::titanic.data$Survived
X <- deepLearnR::titanic.data[,c("Age","SibSp","Fare","Pclass")]
X$Age[is.na(X$Age)] <- mean(X$Age,na.rm=TRUE)
set.seed(512)
inTrain <- sample(1:nrow(X), trunc(nrow(X)*0.8))
X.Train <- X[inTrain,]
Y.Train <- Y[inTrain]
X.Test <- X[-inTrain,]
Y.Test <- Y[-inTrain]
deepLearnR::TensorFlow.Classifier(modelTag="tflr-03",X=X.Train,Y=Y.Train,steps=5000)
pred <- deepLearnR::TensorFlow.predict(modelTag="tflr-03",X=X.Test,Y=Y.Test)
accuracy <- sum(pred == Y.Test)/length(Y.Test)
print(accuracy) # Should be ~ 0.6312849
pred <-  deepLearnR::TensorFlow.predict(modelTag="tflr-03",X=X,Y=Y)
accuracy <- sum(pred == Y)/length(Y)
print(accuracy) # Should be ~ 0.6397306

[1] "tflr-03"

[1] 0.6312849
[1] 0.6397306


In [16]:
library(MASS)
data(Boston)

X <- Boston[,2:14]
y <- Boston[,1]
 
deepLearnR::TensorFlowDNNRegressor(modelTag="tfdnnr-01", X=X, y=y, steps=5000)
pred <- deepLearnR::TensorFlow.regressorEval(modelTag="tfdnnr-01")
mse <- rPython::python.get("mse")
r2 <- rPython::python.get("r2")

[1] "tfdnnr-01"

In [17]:
x.vals <- seq(1:100)
y.vals <- 0.3 + 0.5 * x.vals
lm.tf.fit <- deepLearnR::TensorFlow.SystemLinReg(X = x.vals, Y = y.vals, 
                                     epochs = 100000, learning.rate = .00005)
lm.tf.fit

$slope
[1] 0.5004746

$intercept
[1] 0.2682064

Run skflow/examples/mnist.py

```
(jupyter)[pankracy@piotrek-fedora examples]$ python mnist.py 
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Step #1, avg. loss: 2.64234
Step #101, avg. loss: 1.90145
Step #201, avg. loss: 1.36286
Step #301, avg. loss: 1.10363
Step #401, avg. loss: 0.95203
Step #501, avg. loss: 0.85592
Step #601, epoch #1, avg. loss: 0.78724
Step #701, epoch #1, avg. loss: 0.73279
Step #801, epoch #1, avg. loss: 0.70121
Step #901, epoch #1, avg. loss: 0.65916
Accuracy: 0.860700
Step #1, avg. loss: 3.32872
Step #2001, epoch #3, avg. loss: 1.20425
Step #4001, epoch #7, avg. loss: 0.43666
Step #6001, epoch #10, avg. loss: 0.32565
Step #8001, epoch #14, avg. loss: 0.27147
Step #10001, epoch #18, avg. loss: 0.23350
Step #12001, epoch #21, avg. loss: 0.20529
Step #14001, epoch #25, avg. loss: 0.18313
Step #16001, epoch #29, avg. loss: 0.16682
Step #18001, epoch #32, avg. loss: 0.15348

Accuracy: 0.968500
```

R example
==============

Dataset: https://archive.ics.uci.edu/ml/datasets/Libras+Movement

We try to boost our svm classifier with clustering (hclust). We make hierarchical clustering on our dataset. We cut on level of 40 clusters and try to label our clusters with specific label if every point (of train set) in such cluster have the same label. If this is true, we label our test point with such label. Rest of the points we classify with svm classifier.

We compare this result with standard svm approach.


We can visualize our dataset with help of PCA analysis.
![Plot 2](img/Rplot2.png)
![Plot 3](img/Rplot3.png)

```
library(lattice)
library(class)
library(kernlab)

drawData <- function(Raw, RawData, Classes) {
  temp <- lda(V91 ~ . , data=Raw)
  ldax <- predict(temp, RawData)$x
  xyplot(ldax[,1] ~ ldax[,2] + ldax[,3] + ldax[,4], groups=Classes, pch=19)

  dataMatrix <- as.matrix(RawData)
  rownames(dataMatrix) <- Classes
  pca <- princomp(dataMatrix )
  biplot(pca)

  pc.cr <- prcomp(Raw)
  pca.plot <- xyplot(pc.cr$x[,7] ~ pc.cr$x[,1], groups=Classes)
  pca.plot$xlab <- "First Component"
  pca.plot$ylab <- "Second Component"
  pca.plot
}

getNewTrain <- function(train, classes, clustersNum, verbose) {
  d <- dist(Raw, method = "euclidean")
  fit <- hclust(d, method="ward") 
  groups <- cutree(fit, k=clustersNum)
  if (verbose == TRUE) {
    plot(fit)
    rect.hclust(fit, k=clustersNum, border="red")
  }
  
  for ( i in 1:clustersNum ) {
    cluster <- as.integer(names(groups[groups == i]))
    clusterTrainSubset <- cluster[cluster %in% train]
    if ( length( clusterTrainSubset ) != 0 ) {
      lider <- classes[clusterTrainSubset][1]
      if ( all( classes[clusterTrainSubset] == lider ) ) {
        newItems <- cluster[!cluster %in% clusterTrainSubset]
        if ( length(newItems) != 0) {
          train <- c(train, newItems)
          classes[ newItems ] <- lider
          if ( verbose == TRUE) {
            cat("added items", newItems, "cluster:", cluster, "clusterTrainSubset:", clusterTrainSubset, "to class", lider,"\n")
          }
        }
      }
    }
  }
  return (list("train" = train, "classes" = classes))
}

getResults <- function(Raw, RawData, Classes, train, clustersNum, maxPcaDim, verbose) {
  filter <- ksvm(V91~.,data=Raw[train,],kernel="laplacedot",kpar=list(sigma=0.05),C=5,cross=3)
  result1 <- predict(filter, Raw[-train,-91])
  table1 <- table(round(result1), Raw[-train,91])

  pc <- prcomp(RawData)
  pc$x
  transformed <- as.matrix(pc$x[,1:maxPcaDim])
  transformed <- cbind(Classes, transformed)
  
  newTrain <- getNewTrain(train, Classes, clustersNum, verbose)
  trainDiff <- newTrain$train[! newTrain$train %in% train]
  
  filter <- ksvm(Classes~.,data=transformed[newTrain$train,],kernel="laplacedot",kpar=list(sigma=0.05),C=5,cross=3)
  result2 <- predict(filter, as.matrix(transformed[-newTrain$train,-1]))
  table2 <- table( c(round(result2), newTrain$classes[trainDiff]), c(transformed[-newTrain$train,1], Classes[trainDiff]) )

  return (c(sum(diag(table1))/sum(table1), sum(diag(table2))/sum(table2)))
}

drawResults <- function(Raw, RawData, Classes, classesDim, pcaDim, clustersNum, trainRatio, verbose) {
  result <- matrix(0, ncol=2, nrow=5)
  for ( j in 1:5 ) {
    for ( k in 1:5 ) {
      train <- vector()
      for ( i in 1:classesDim ) {
        buffer <- rownames(Raw[Raw[,91] == i,])
        train <- as.integer(c(train, sample(buffer, round(length(buffer) * (trainRatio + k/10)))))
      }
      result[k,] <- result[k,] + getResults(Raw, RawData, Classes, train, clustersNum, pcaDim, verbose)
    }
  }
  print(result <- result / 5)

  plot(1:5, result[,1], type="o", col="blue", ylim=c(0,1))
  points(1:5,result[,2], type="o", col="red")
}

Raw <- read.csv("libras_all.data",header=FALSE)

classesDim <- 10
pcaDim <- 10
clustersNum <- 40
trainRatio <- 10/100

Raw <- Raw[Raw[,91] %in% 1:classesDim,]
RawData <- Raw[,1:90]
Classes <- Raw[,91]

drawResults(Raw, RawData, Classes, classesDim, pcaDim, clustersNum, trainRatio, FALSE)

drawData(Raw, RawData, Classes)
```

Results
===============

Red line is svm supported by clustering. Blue line is standard svm approach. We can see that svm supported with unsupervised learning 

![Plot 1](img/Rplot1.png)
